In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("MainDataSet1D.csv").tail(1000)

In [120]:
df['candle_direction'] = 'None'

# Calculate shadow lengths
upper_shadow_length = df["Day_High"] - df[["Day_Close", "Day_Open"]].max(axis=1)
lower_shadow_length = df[["Day_Close", "Day_Open"]].min(axis=1) - df["Day_Low"]

# Check conditions
green_condition = df["Day_Close"] > df["Day_Open"]
upper_shadow_condition = upper_shadow_length < lower_shadow_length * 0.5
lower_shadow_condition = lower_shadow_length < upper_shadow_length * 0.5

# Update candle_direction column
df.loc[green_condition & upper_shadow_condition, 'candle_direction'] = 'candle up'
df.loc[green_condition & lower_shadow_condition, 'candle_direction'] = 'candle down'


In [121]:
df = df[df["candle_direction"]== "candle up"]
df.iloc[-1]

Day_Open                       22475.55
Day_High                        22655.8
Day_Low                         22441.9
Day_Close                       22643.4
Day_Trend                             1
Day_Date                     2024-04-29
Day_SwingMin                    21137.2
Day_SwingMax                    22775.7
Datetime            2024-04-29 05:30:00
Open                           22475.55
High                            22655.8
Low                             22441.9
Close                           22643.4
Pivot                      22580.366667
Candle                            Green
5EMA                       22489.752884
15EMA                      22402.337313
20EMA                      22377.529983
50EMA                      22185.688777
200EMA                     20891.254813
RSI                           65.426769
Trend                                 1
CandleBody                       167.85
Small_Swing_Min                 21137.2
Small_Swing_Max                 22775.7


In [157]:
open_order = None
sl_order = None
tg_order = None
side = None

Tradbook = []
def backtest(row):
    global points,open_order,sl_order,tg_order,side
    print(f" POSITION ------- ORDER:{open_order} | POINTS:{points} | SL:{sl_order} | TG :{tg_order} ")
    if row["candle_direction"] != "None" and open_order is None:
        open_order = row["Close"]
        side = "CE" if  row["candle_direction"] == "candle up" else  "PE"
        # sl_order = row["Small_Swing_Min"] if  row["candle_direction"] == "candle up" else  row["Small_Swing_Max"]
        sl_order = row["Low"] if  row["candle_direction"] == "candle up" else  row["High"]
        tg_order = open_order + 100
        print("BUY AT ")
    if side == "CE" and open_order != None and row["Close"] >= tg_order :
        order_stack = {"Buy At":open_order,"Sell At":row["Close"], "Side":"CE","PnL":row["Close"] -open_order}
        print("-----------------------------------------------------------")
        print("CLOSE With PROFIT :",order_stack)
        print("-----------------------------------------------------------")
        Tradbook.append(order_stack)
        open_order = None
        sl_order= None
        tg_order = None
    if side == "CE" and open_order != None and row["Close"] <= sl_order :
        order_stack = {"Buy At":open_order,"Sell At":row["Close"], "Side":"CE","PnL":row["Close"] -open_order}
        print("-----------------------------------------------------------")
        print("CLOSE With LOSS :",order_stack)
        print("-----------------------------------------------------------")
        Tradbook.append(order_stack)
        open_order = None
        sl_order= None
        tg_order = None
    
        
        # print(f"OPEN ORDER------:{open_order} | SL:{sl_order} | TG :{tg_order} ")
df.apply(backtest,axis=1)
    

 POSITION ------- ORDER:None | POINTS:0 | SL:None | TG :None 
BUY AT 
 POSITION ------- ORDER:17536.2 | POINTS:0 | SL:17351.7 | TG :17636.2 
 POSITION ------- ORDER:17536.2 | POINTS:0 | SL:17351.7 | TG :17636.2 
-----------------------------------------------------------
CLOSE With LOSS : {'Buy At': 17536.2, 'Sell At': 16955.4, 'Side': 'CE', 'PnL': -580.7999999999993}
-----------------------------------------------------------
 POSITION ------- ORDER:None | POINTS:0 | SL:None | TG :None 
BUY AT 
 POSITION ------- ORDER:17086.2 | POINTS:0 | SL:16833.2 | TG :17186.2 
-----------------------------------------------------------
CLOSE With PROFIT : {'Buy At': 17086.2, 'Sell At': 17805.2, 'Side': 'CE', 'PnL': 719.0}
-----------------------------------------------------------
 POSITION ------- ORDER:None | POINTS:0 | SL:None | TG :None 
BUY AT 
 POSITION ------- ORDER:17925.2 | POINTS:0 | SL:17748.8 | TG :18025.2 
 POSITION ------- ORDER:17925.2 | POINTS:0 | SL:17748.8 | TG :18025.2 
--------

6      None
17     None
25     None
28     None
34     None
       ... 
574    None
593    None
596    None
601    None
607    None
Length: 123, dtype: object